# Configuracion de grafica a usar

In [1]:
import os
os.environ["CUDA_DEVICE_ORDER"]="PCI_BUS_ID";
 
# lA ID de la GPU a usar, puede ser desde 0 hasta las N GPU's. Si es -1 significa que es en la CPU
os.environ["CUDA_VISIBLE_DEVICES"]="1";

# Importacion de librerias

In [3]:
from __future__ import absolute_import, division, print_function, unicode_literals
import tensorflow as tf
from tensorflow import keras

%matplotlib inline
import sys
sys.path.append('../')
from Datasets_utils.DatasetsLoader import VideoDataGenerator

# Configuraciones para Tensorflow y Keras

In [4]:
print("Num GPUs Available: ", len(tf.config.experimental.list_physical_devices('GPU')))

Num GPUs Available:  1


In [5]:
gpus = tf.config.experimental.list_physical_devices('GPU')
tf.config.experimental.set_memory_growth(gpus[0], True)

In [6]:
tf.debugging.set_log_device_placement(True)

In [7]:
#Comprobar que estoy ejecutandome en modo eagerly
tf.executing_eagerly()

True

# Carga de Datos

In [ ]:
root_path = "/home/jefelitman/DataSets/ucf101/split_1"
root_path

In [ ]:
batch = 30
original_size = [171,128]
size = [112,112]
frames = 16
canales = 3

In [ ]:
dataset = VideoDataGenerator(directory_path = root_path, 
                             batch_size = batch, 
                             original_frame_size = original_size, 
                             frame_size=size, 
                             video_frames = frames, 
                             temporal_crop = ('sequential', None), 
                             frame_crop = ('random', 4), 
                             shuffle = True, 
                             conserve_original = True)

# Red Neuronal LTC

### Construccion del modelo

In [ ]:
#Entrada de la red neuronal
video_shape = tuple([frames]+size+[canales])
dropout = 0.5
entrenamiento = True

In [ ]:
entrada = keras.Input(shape=video_shape,
                     batch_size=batch,
                     name="Entrada video")
#Conv1
x = keras.layers.Conv3D(filters=64, kernel_size=3, activation="relu", use_bias=False)(entrada)
x = keras.layers.MaxPool3D(pool_size=(1,2,2),strides=(2,2,2))(x)

#Conv2
x = keras.layers.Conv3D(filters=128, kernel_size=3, activation="relu", use_bias=False)(x)
x = keras.layers.MaxPool3D(pool_size=(2,2,2),strides=(2,2,2))(x)

#Conv3
x = keras.layers.Conv3D(filters=256, kernel_size=3, activation="relu", use_bias=False)(x)
x = keras.layers.MaxPool3D(pool_size=(2,2,2),strides=(2,2,2))(x)

#Conv4
x = keras.layers.Conv3D(filters=256, kernel_size=3, activation="relu", use_bias=False)(x)
x = keras.layers.MaxPool3D(pool_size=(2,2,2),strides=(2,2,2))(x)

#Conv5
x = keras.layers.Conv3D(filters=256, kernel_size=3, activation="relu", use_bias=False)(x)
x = keras.layers.MaxPool3D(pool_size=(2,2,2),strides=(1,1,1))(x)

#fc6
x = tf.reshape(tensor=x, shape=[batch,-1], name="Aplanado")
x = keras.layers.Dense(2048, activation="relu")(x)
x = keras.layers.Dropout(dropout)(x, training=entrenamiento)

#fc7
x = keras.layers.Dense(2048, activation="relu")(x)
x = keras.layers.Dropout(dropout)(x, training=entrenamiento)

#fc8
salidas= keras.layers.Dense(len(dataset.to_class), activation="softmax")(x)

ltc = keras.Model(entrada, salidas, name="LTC")
ltc.summary()

In [ ]:
keras.utils.plot_model(ltc, 'LTC.png', show_shapes=True)

### Entrenamiento de la red